In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ast
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from pprint import pprint
from collections import Counter
from galleries_mapping import *
import matplotlib.pyplot as plt
from sklearn.utils import resample

In [3]:
def plot_tags_count(data: pd.DataFrame | dict):
    if isinstance(data, pd.DataFrame):
        data = Counter([item for sublist in data['labels'] for item in sublist])
    sorted_values_cntr = {k: v for k, v in sorted(data.items(), key=lambda item: item[1], reverse=True)}
    plt.figure(figsize=(10, 10))
    plt.pie(
        sorted_values_cntr.values(),
        labels=sorted_values_cntr.keys(),
        autopct='%1.1f%%', startangle=0
    )
    plt.axis('equal')
    plt.title('Label Distribution in Test Set')

    plt.show()


def plot_variance_per_key(data: dict):
    plt.figure(figsize=(10, 10))
    plt.bar(data.keys(), data.values())
    plt.title('Label Distribution in Test Set')
    plt.show()

In [4]:
SRC_DIR = Path('/Volumes/external_drive')
FILTERED_PORNHUB_CATEGORIES = [
    'anal',
    'bbw',
    'big ass',
    'big dick',
    'big tits',
    'blonde',
    'blowjob',
    'bondage',
    'brunette',
    'cosplay',
    'creampie',
    'cumshot',
    'double penetration',
    'ebony',
    'feet',
    'fingering',
    'fisting',
    'handjob',
    'hardcore',
    'lesbian',
    'massage',
    'masturbation',
    'milf',
    'old/young',
    'pissing',
    'public',
    'pussy licking',
    'red head',
    'rough sex',
    'small tits',
    'smoking',
    'solo',
    'squirt',
    'strap on',
    'striptease',
    'tattooed women',
    'teen',
    'threesome',
    'toys',
    'transgender'
]

try:
    _df = pd.read_csv('datasets/images_high_res_dataset.csv')
except FileNotFoundError:
    _df = pd.read_csv(SRC_DIR / 'images_high_res_dataset.csv')

print("Dataset loaded.")
_df.drop(['models'], axis=1, inplace=True)
_df['categories'] = _df['categories'].apply(ast.literal_eval)
_df['categories_suggestions'] = _df['categories_suggestions'].apply(ast.literal_eval)
print("Parsed list columns categories")

Dataset loaded.
Parsed list columns categories


In [5]:
df = _df.copy(True)
df.shape

(1345430, 7)

# Merge suggestions and categories

In [7]:
def merge_categories(row):
    categories = set(row['categories'])
    categories_suggestions = set(row['categories_suggestions'])
    categories_superset = {category.strip().lower() for category in categories.union(categories_suggestions)}
    return list(categories_superset)


df['labels'] = df['categories'].apply(lambda x: [category.strip().lower() for category in x])
df['labels'] = df.apply(merge_categories, axis=1)
df.drop(['categories_suggestions', 'categories'], axis=1, inplace=True)
df.shape

(1345430, 6)

# Purge nationalities from tags

In [8]:
nationality_tags_to_purge = {
    'african',
    'american',
    'arab',
    'argentina',
    'australian',
    'brazilian',
    'british',
    'canadian',
    'chinese',
    'colombian',
    'cuban',
    'czech',
    'dutch',
    'european',
    'filipina',
    'french',
    'german',
    'hungarian',
    'indian',
    'italian',
    'japanese',
    'korean',
    'mexican',
    'pinay',
    'polish',
    'russian',
    'spanish',
    'thai',
    'ukrainian',
    'venezuela',
    'white'
}
nationality_tags_to_purge = {n.lower() for n in nationality_tags_to_purge}
df['labels'] = df.labels.apply(lambda x: list(set(x) - nationality_tags_to_purge))
df.shape

(1345430, 6)

# Apply gallery mapping

In [9]:
def gallery_mapping(row):
    labels = row.labels

    out = []
    for L in labels:
        _fetched = GALLERIES_MAP.get(L, None)
        if _fetched is remove_tag:
            continue
        elif _fetched is remove_gallery:
            return None
        elif isinstance(_fetched, list):
            out.extend(_fetched)
        elif _fetched is keep_tag:
            out.append(L.lower())

    return list(set(out))


df['labels'] = df.apply(gallery_mapping, axis=1)
df = df[df['labels'].notnull()]
df.shape

(1114113, 6)

# Dataframe clean up

In [10]:
df['file_path'] = df['gallery_category'] + '/' + df['gallery_name'] + '/' + df['filename']
df = df.drop(
    ['gallery_category', 'gallery_name', 'filename'], axis=1
)
df.reset_index(inplace=True, drop=True)

In [11]:
df.drop(
    ['height', 'width'], axis=1, inplace=True
)

In [12]:
df.shape

(1114113, 2)

In [14]:
df.to_csv('datasets/all_file_paths.csv')

# Balancing the dataset

In [15]:
label_counts = Counter(item for sublist in df['labels'] for item in sublist)
ascending_labels = [
    k for k, v in sorted(label_counts.items(), key=lambda item: item[1])
]
label_proportions = {
    k: v / len(df) for k, v in sorted(label_counts.items(), key=lambda item: item[1])
}

In [21]:
def get_balanced_df(og_df, max_samples=1000):
    def get_rows_with_label(dataframe, label):
        return dataframe[dataframe['labels'].apply(lambda x: label in x)]

    balanced_dfs = {label: pd.DataFrame() for label in label_counts.keys()}

    for label in tqdm(ascending_labels, total=len(ascending_labels), desc='Balancing dataset'):
        label_df = get_rows_with_label(og_df, label)

        label_df = resample(
            label_df,
            n_samples=max_samples,
            random_state=42
        )

        balanced_dfs[label] = label_df

    balanced_df = pd.concat(balanced_dfs).drop_duplicates(subset='file_path').reset_index(drop=True)

    return balanced_df

In [22]:
small_balanced_df = get_balanced_df(df, max_samples=1000)
medium_balanced_df = get_balanced_df(df, max_samples=3000)
final_balanced_df = df.copy()

Balancing dataset: 100%|██████████| 113/113 [00:30<00:00,  3.72it/s]


In [23]:
small_balanced_df.to_csv('datasets/small_file_paths.csv')
medium_balanced_df.to_csv('datasets/medium_file_paths.csv')
final_balanced_df.to_csv('datasets/all_file_paths.csv')

# One hot encoding

In [24]:
def dataframe_one_hot_encoding(dataframe: pd.DataFrame):
    all_labels = set(label for labels in dataframe['labels'] for label in labels)
    one_hot_encoded = pd.DataFrame()

    for label in tqdm(all_labels, total=len(all_labels), desc='One hot encoding'):
        one_hot_encoded[label] = dataframe['labels'].apply(lambda x: 1 if label in x else 0)

    final_df = pd.concat([dataframe, one_hot_encoded], axis=1)
    final_df = final_df.drop(columns=['labels', ], axis=1)
    final_df.reset_index(drop=True, inplace=True)

    cols_sorted = sorted(list(final_df.columns))
    cols_sorted.remove("file_path")

    final_df = final_df[["file_path", *cols_sorted]]

    return final_df

In [25]:
small_final_df = dataframe_one_hot_encoding(small_balanced_df)
medium_final_df = dataframe_one_hot_encoding(medium_balanced_df)
final_df = dataframe_one_hot_encoding(df)

One hot encoding:  88%|████████▊ | 100/113 [00:06<00:00, 13.99it/s]/var/folders/cq/qnm4dkj53rq748w0d5l1yz5h0000gn/T/ipykernel_46410/3430957949.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  one_hot_encoded[label] = dataframe['labels'].apply(lambda x: 1 if label in x else 0)
/var/folders/cq/qnm4dkj53rq748w0d5l1yz5h0000gn/T/ipykernel_46410/3430957949.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  one_hot_encoded[label] = dataframe['labels'].apply(lambda x: 1 if label in x else 0)
One hot encoding:  90%|█████████ | 102/11

In [26]:
small_final_df.to_csv("datasets/small_one_hot.csv")
medium_final_df.to_csv("datasets/medium_one_hot.csv")
final_df.to_csv("datasets/full_one_hot.csv")

# AI GEN

In [27]:
df.shape

(1114113, 2)

In [28]:
from galleries_mapping import *

In [29]:
_mandatory_columns = [
    'age',
]
mandatory_columns = list(
    x.strip() for x in AI_GEN_TAGS[_mandatory_columns].values.flatten() if isinstance(x, str)
)
_negative_columns = [
    'negative', 'more_people'
]
negative_columns = list(
    x.strip() for x in AI_GEN_TAGS[_negative_columns].values.flatten() if isinstance(x, str)
)
tags_to_drop = {
    'blowjob', 'reality', 'hardcore', 'cumshot', 'cowgirl', 'ass fucking', 'doggy style', 'teen', 'lesbian', 'model',
    'big cock', 'anal', 'bdsm', 'bondage', 'fetish', 'face', 'kissing', 'handjob', 'facial', 'fingering', 'groupsex',
    'cum in mouth', 'old young', 'titjob', 'interracial', 'pussy licking', 'threesome', 'pov', 'femdom', 'christmas',
    'girlfriend', 'cosplay', 'facesitting', 'massage', 'deepthroat', 'strapon', 'cheating', 'humping', 'cum in pussy',
    'ass licking', 'creampie', 'ball licking', 'spanking', 'orgasm', 'double penetration', 'couple', 'family',
    'anal gape', 'bbc', 'party', 'schoolgirl', 'fisting', 'missionary', 'squirting', 'pissing', 'gangbang', 'old man',
    'ffm', 'cuckold', 'seduction', 'tribbing', 'orgy', 'flexible', 'cfnm', 'footjob', 'blowbang', 'pegging', 'pregnant',
    'swingers', 'gloryhole', 'caught', 'college', 'yoga', 'casting', 'stripper', 'step sister', 'voyeur', 'mmf',
    'bukkake', 'gyno', 'small cock', 'babysitter', 'cheerleader', 'cum swapping', 'bisexual', 'goth', 'braces', 'pawg',
    'pretty', 'pigtails', 'emo', 'latex', 'babe', 'step brother', 'twink', 'shemale', 'ballerina', 'twins', 'pornstar',
    'model', 'latex', 'emo', 'latex', 'babe', 'leather', 'pigtails', 'halloween', 'wedding',
    'tall', 'doctor', 'vintage'
}

In [30]:
def filter_positives(row):
    labels = row.labels

    for L in labels:
        if L in mandatory_columns:
            return labels


def filter_negatives(row):
    labels = row.labels
    for L in labels:
        if L in negative_columns:
            # print(f"Label {L} in negative_columns.")
            return None
        if L in tags_to_drop:
            # print(f"Label {L} in tags_to_drop.")
            return None
        # print(f"Label {L} is allowed.")
    return labels


ai_df = df.copy()
ai_df['labels'] = ai_df.apply(filter_positives, axis=1)
ai_df = ai_df[ai_df['labels'].notnull()]
print(ai_df.shape)

ai_df['labels'] = ai_df.apply(filter_negatives, axis=1)
ai_df = ai_df[ai_df['labels'].notnull()]
print(ai_df.shape)

(332966, 2)
(80250, 2)


In [31]:
ai_df.reset_index(inplace=True, drop=True)

In [32]:
ai_df['labels_length'] = ai_df['labels'].apply(len)
ai_df = ai_df[ai_df['labels_length'] > 5]
ai_df.drop(['labels_length'], axis=1, inplace=True)
ai_df.reset_index(drop=True, inplace=True)
ai_df.to_csv('datasets/ai_gen_dataset_5_cats.csv')
print("5 cats df shape:", ai_df.shape)

ai_df['labels_length'] = ai_df['labels'].apply(len)
ai_df = ai_df[ai_df['labels_length'] > 10]
ai_df.drop(['labels_length'], axis=1, inplace=True)
ai_df.reset_index(drop=True, inplace=True)
ai_df.to_csv('datasets/ai_gen_dataset_10_cats.csv')
print("10 cats df shape:", ai_df.shape)

5 cats df shape: (27112, 2)
10 cats df shape: (10524, 2)


In [33]:
all_labels = [label for labels in ai_df['labels'] for label in labels]
cntr = Counter(all_labels)
cntr

Counter({'milf': 8179,
         'big tits': 7803,
         'spreading': 7304,
         'pussy': 6994,
         'legs': 5795,
         'clothed': 5713,
         'panties': 5618,
         'heels': 5146,
         'nipples': 4944,
         'mature': 4704,
         'close up': 4468,
         'solo': 4224,
         'brunette': 4184,
         'thick': 4079,
         'curvy': 4079,
         'chubby': 4079,
         'bbw': 4079,
         'masturbation': 4031,
         'skirt': 3964,
         'blonde': 3659,
         'lingerie': 3588,
         'dildo': 3313,
         'sex toys': 3313,
         'undressing': 3193,
         'stockings': 3077,
         'shaved': 2831,
         'hairy': 2258,
         'upskirt': 2201,
         'feet': 1472,
         'outdoor': 1347,
         'tattoo': 1288,
         'redhead': 1276,
         'tiny tits': 1238,
         'armpit': 1237,
         'glasses': 1196,
         'uniform': 820,
         'pantyhose': 792,
         'smoking': 734,
         'dress': 697,
       

In [34]:
ai_df.head()

,labels,file_path
0,"[heels, legs, nipples, clothed, shaved, close ...",big-tits/super-busty-blonde-mary-carey-lets-he...
1,"[heels, legs, nipples, clothed, shaved, close ...",big-tits/super-busty-blonde-mary-carey-lets-he...
2,"[heels, legs, nipples, clothed, shaved, close ...",big-tits/super-busty-blonde-mary-carey-lets-he...
3,"[heels, legs, nipples, clothed, shaved, close ...",big-tits/super-busty-blonde-mary-carey-lets-he...
4,"[heels, legs, nipples, clothed, shaved, close ...",big-tits/super-busty-blonde-mary-carey-lets-he...
